In [1]:
%%capture
import tensorflow as tf
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
tf.__version__

## Define tfkeras model

Only tfkeras tf-2.0.0+😉

In [2]:
def MobileBlock(x, inter_filters=96, out_filters=16, kernel_size=5, strides=1, skip=False):     
    
    # 1x1 
    x_r = Conv2D(filters=inter_filters, kernel_size=1, padding='same')(x) 
    x_r = ReLU(6.0)(x_r)
    
    # dw
    x_r = DepthwiseConv2D(kernel_size=kernel_size, strides=strides, padding='same')(x_r)
    x_r = ReLU(6.0)(x_r)
    
    # 1x1 
    x_r = Conv2D(filters=out_filters, kernel_size=1, padding='same', activation=None)(x_r)   

    if skip:
        return Add()([x, x_r])
    else:
        return x_r
    

In [3]:
def Hand3D(input_shape):

    inputs = tf.keras.layers.Input(shape=input_shape, name='input')

    # first block
    x = Conv2D(filters=24, kernel_size=3, strides=2, padding='same')(inputs)   
    x = ReLU(6.0)(x)
    x = DepthwiseConv2D(kernel_size=3, strides=1, padding='same')(x)
    x = ReLU(6.0)(x)
    x = Conv2D(filters=8, kernel_size=1, padding='same', activation=None)(x)   
            
  
    x = MobileBlock(x, inter_filters=32, out_filters=16, kernel_size=3, strides=2)
    x = MobileBlock(x, inter_filters=96, out_filters=16, kernel_size=3, skip=True)
    
    x = MobileBlock(x, inter_filters=96, out_filters=24, kernel_size=5,  strides=2)
    x = MobileBlock(x, inter_filters=144, out_filters=24, kernel_size=5, skip=True)
    
    x = MobileBlock(x, inter_filters=144, out_filters=40, kernel_size=3, strides=2)
    x = MobileBlock(x, inter_filters=240, out_filters=40, kernel_size=3, skip=True)    
    x = MobileBlock(x, inter_filters=240, out_filters=40, kernel_size=3, skip=True)
    
    x = MobileBlock(x, inter_filters=240, out_filters=56, kernel_size=5, strides=1)
    x = MobileBlock(x, inter_filters=336, out_filters=56, kernel_size=5, skip=True)    
    x = MobileBlock(x, inter_filters=336, out_filters=56, kernel_size=5, skip=True)
    
    x = MobileBlock(x, inter_filters=336, out_filters=96, kernel_size=5, strides=2)
    x = MobileBlock(x, inter_filters=576, out_filters=96, kernel_size=5, skip=True)    
    x = MobileBlock(x, inter_filters=576, out_filters=96, kernel_size=5, skip=True)
    x = MobileBlock(x, inter_filters=576, out_filters=96, kernel_size=5, skip=True)
    
    
    # ending block
    x = Conv2D(filters=576, kernel_size=1, strides=1, padding='same')(x)   
    x = ReLU(6.0)(x)
    x = DepthwiseConv2D(kernel_size=3, strides=1, padding='same')(x)
    x = ReLU(6.0)(x)
    x = DepthwiseConv2D(kernel_size=3, strides=2, padding='same')(x)
    x = ReLU(6.0)(x)
    x = Conv2D(filters=144, kernel_size=1, padding='same', activation=None)(x)    
    x = Flatten()(x)

    # output head
    handflag = Dense(1, activation='sigmoid', name='handflag')(x)
    handedness = Dense(1, activation='sigmoid', name='handedness')(x)    
    landmarks = Dense(63, name='landmarks')(x)

    
    model = tf.keras.models.Model(inputs=inputs, outputs=[handflag, handedness, landmarks])
    return model


In [4]:
INPUT_SHAPE = (224, 224,3)
model = Hand3D(INPUT_SHAPE)

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 24) 672         input[0][0]                      
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 112, 112, 24) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
depthwise_conv2d (DepthwiseConv (None, 112, 112, 24) 240         re_lu[0][0]                      
______________________________________________________________________________________________

In [10]:
model.save('model/new_blank.h5')

In [16]:
model.save('model/saved_model',save_format='tf')

## TFLite Weights convert.

In [6]:
import glob, os
import numpy as np

In [7]:
np_files = glob.glob("model/w/*.npy")
weights_dict = {}
for f in np_files:
    name = os.path.split(f)[-1].replace(".kernel", "/kernel:0").replace(".bias", "/bias:0").replace(".npy", "")
    if "depthwise" in name:
        name = name.replace("/kernel:0", "/depthwise_kernel:0")    
    
    weight = np.load(f)
    weights_dict[name] = weight
weights_dict.keys()


dict_keys(['conv2d/bias:0', 'conv2d/kernel:0', 'conv2d_1/bias:0', 'conv2d_1/kernel:0', 'conv2d_10/bias:0', 'conv2d_10/kernel:0', 'conv2d_11/bias:0', 'conv2d_11/kernel:0', 'conv2d_12/bias:0', 'conv2d_12/kernel:0', 'conv2d_13/bias:0', 'conv2d_13/kernel:0', 'conv2d_14/bias:0', 'conv2d_14/kernel:0', 'conv2d_15/bias:0', 'conv2d_15/kernel:0', 'conv2d_16/bias:0', 'conv2d_16/kernel:0', 'conv2d_17/bias:0', 'conv2d_17/kernel:0', 'conv2d_18/bias:0', 'conv2d_18/kernel:0', 'conv2d_19/bias:0', 'conv2d_19/kernel:0', 'conv2d_2/bias:0', 'conv2d_2/kernel:0', 'conv2d_20/bias:0', 'conv2d_20/kernel:0', 'conv2d_21/bias:0', 'conv2d_21/kernel:0', 'conv2d_22/bias:0', 'conv2d_22/kernel:0', 'conv2d_23/bias:0', 'conv2d_23/kernel:0', 'conv2d_24/bias:0', 'conv2d_24/kernel:0', 'conv2d_25/bias:0', 'conv2d_25/kernel:0', 'conv2d_26/bias:0', 'conv2d_26/kernel:0', 'conv2d_27/bias:0', 'conv2d_27/kernel:0', 'conv2d_28/bias:0', 'conv2d_28/kernel:0', 'conv2d_29/bias:0', 'conv2d_29/kernel:0', 'conv2d_3/bias:0', 'conv2d_3/kern

In [8]:
count = 0
for each_layer in model.layers:    
    if hasattr(each_layer, 'depthwise_kernel') and each_layer.depthwise_kernel is not None:    
        K.set_value(each_layer.weights[0], weights_dict[each_layer.name+'/depthwise_kernel:0'].transpose(1,2,3,0))
        print(each_layer.name+'.weight')
        count += 1
    
    if hasattr(each_layer, 'kernel') and each_layer.kernel is not None:        
        if len(weights_dict[each_layer.name+'/kernel:0'].shape) == 4:   
            K.set_value(each_layer.weights[0], weights_dict[each_layer.name+'/kernel:0'].transpose(1,2,3,0))            
            
        else:
            K.set_value(each_layer.weights[0], weights_dict[each_layer.name+'/kernel:0'].transpose(1,0))
            
        print(each_layer.name+'.weight')
        count += 1
    if hasattr(each_layer, 'bias') and each_layer.bias is not None:
        K.set_value(each_layer.weights[1], weights_dict[each_layer.name+'/bias:0'])
        print(each_layer.name+'.bias')
        count += 1
    
    
   

conv2d.weight
conv2d.bias
depthwise_conv2d.weight
depthwise_conv2d.bias
conv2d_1.weight
conv2d_1.bias
conv2d_2.weight
conv2d_2.bias
depthwise_conv2d_1.weight
depthwise_conv2d_1.bias
conv2d_3.weight
conv2d_3.bias
conv2d_4.weight
conv2d_4.bias
depthwise_conv2d_2.weight
depthwise_conv2d_2.bias
conv2d_5.weight
conv2d_5.bias
conv2d_6.weight
conv2d_6.bias
depthwise_conv2d_3.weight
depthwise_conv2d_3.bias
conv2d_7.weight
conv2d_7.bias
conv2d_8.weight
conv2d_8.bias
depthwise_conv2d_4.weight
depthwise_conv2d_4.bias
conv2d_9.weight
conv2d_9.bias
conv2d_10.weight
conv2d_10.bias
depthwise_conv2d_5.weight
depthwise_conv2d_5.bias
conv2d_11.weight
conv2d_11.bias
conv2d_12.weight
conv2d_12.bias
depthwise_conv2d_6.weight
depthwise_conv2d_6.bias
conv2d_13.weight
conv2d_13.bias
conv2d_14.weight
conv2d_14.bias
depthwise_conv2d_7.weight
depthwise_conv2d_7.bias
conv2d_15.weight
conv2d_15.bias
conv2d_16.weight
conv2d_16.bias
depthwise_conv2d_8.weight
depthwise_conv2d_8.bias
conv2d_17.weight
conv2d_17.bias
co

In [9]:
count

104